## 문제 질문 1
- mtcars데이터셋(data/mtcars.csv)의 qsec컬럼을 최소최대 척도(Min-Max Scale)로 변환하 후 0.5보다 큰 값을 가지는 레코드 수를 구하시오.


In [ ]:
import pandas as pd
a = pd.read_csv('data/mtcars.csv', index_col=0)

# 사용자 코딩
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(a[['qsec']])
a['normalized_qsec'] = scaler.transform(a[['qsec']])
print(a['normalized_qsec'])
print(len(a[a['normalized_qsec'] > 0.5]))

In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# 사용자 코딩
# 환불 금액 0으로 채우기
X_train['환불금액'] = X_train['환불금액'].fillna(0)
X_test['환불금액'] = X_test['환불금액'].fillna(0)
numerical_list = ['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']
scaler = MinMaxScaler()
scaler.fit(X_train[numerical_list])
X_train[numerical_list]= scaler.transform(X_train[numerical_list])
X_test[numerical_list]= scaler.transform(X_test[numerical_list])
category = ['주구매상품', '주구매지점']

from sklearn.preprocessing import LabelEncoder
for cat in category:
	le = LabelEncoder()
	X_train[cat] = le.fit_transform(X_train[cat])
	X_test[cat] = le.transform(X_test[cat])

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

train_x, test_x, train_y, test_y = train_test_split(X_train.drop(['cust_id'], axis=1), y_train.drop(['cust_id'], axis=1), test_size=0.2)
model1 = RandomForestClassifier(n_estimators=100, max_depth=5)
model1.fit(train_x, train_y)
pred1 = model1.predict_proba(test_x)
print(test_y)
print(roc_auc_score(test_y, pred1[:, 1:]))

model2 = AdaBoostClassifier()
model2.fit(train_x, train_y)
pred2 = model2.predict_proba(test_x)
print(roc_auc_score(test_y, pred2[:, 1:]))

model3 = VotingClassifier(estimators=[('rf', model1), ('ad', model2)], voting='soft')
model3.fit(train_x, train_y)
pred3 = model3.predict_proba(test_x)
print(roc_auc_score(test_y, pred3[:, 1:]))



model1 = RandomForestClassifier(n_estimators=100, max_depth=5)
idxs = X_test['cust_id']
X_train = X_train.drop(['cust_id'], axis=1)
y_train = y_train.drop(['cust_id'], axis=1)
X_test = X_test.drop(['cust_id'], axis=1)

model1.fit(X_train, y_train)
pred1 = model1.predict_proba(X_test)

model2 = AdaBoostClassifier()
model2.fit(X_train, y_train)
pred2 = model2.predict_proba(X_test)

model3 = VotingClassifier(estimators=[('rf', model1), ('ad', model2)], voting='soft')
model3.fit(X_train, y_train)
pred3 = model3.predict_proba(X_test)

pd.DataFrame({'id':idxs, 'pred':pred3[:,1]}).to_csv('003000000.csv', index=False)
check = pd.read_csv('003000000.csv')
print(check.head())
# print(X_train.head())
# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)
